In [537]:
from contextlib import contextmanager
from threading  import Lock, Thread
from time import sleep

class SharedExclusive(object):

    def __init__(self):
        self.write_lock = False
        self.write_lock_obj = Lock()
        self.num_r = 0

    def r_acquire(self):
        print("\nRead_lock Called:")
        if self.write_lock:
            print("Wait for write_lock")
        else:
            print("Read lock")
            self.num_r += 1

    def r_release(self):
        if self.num_r > 0:
            print("Release read lock")
            self.num_r -= 1

    @contextmanager
    def r_locked(self):
        try:
            self.r_acquire()
            yield
        finally:
            self.r_release()

    def w_acquire(self):
        print("\nWrite_lock Called:")
        if self.num_r:
            print("Wait for read_lock")
        elif self.write_lock:
            print("Wait for write_lock")
        else:
            print('Write lock')
            self.write_lock = True
            self.write_lock_obj.acquire()

    def w_release(self):
        if self.write_lock:
            print('Release write lock')
            self.write_lock = False
            self.write_lock_obj.release()

    @contextmanager
    def w_locked(self):
        try:
            self.w_acquire()
            yield
        finally:
            self.w_release()

In [541]:
lock = SharedExclusive()
x = 0
def write(num, lock):
    global x
    with lock.w_locked():
        local_var = x
        local_var += num
        x = local_var

def read(lock):
    global x
    with lock.r_locked():
        print(x)

t1 = Thread(target=read, args=(lock,))
t2 = Thread(target=write, args=(10, lock))
t3 = Thread(target=read, args=(lock,))
t4 = Thread(target=write, args=(20, lock))
t5 = Thread(target=write, args=(30, lock))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()

print(f'The final counter is {x}')


Read_lock Called:
Write_lock Called:
Read lock
0
Release read lock

Write lock
Release write lock

Read_lock Called:
Write_lock Called:
Read lock
10
Release read lock

Write lock
Release write lock

Write_lock Called:
Write lock
Release write lock
The final counter is 60
